#### OBJETIVO: Contruir modelos para predecir PM2.5
- Esta notebook se hizo antes que la 02

In [22]:
#Librerias
import pandas as pd
import numpy as np

from datetime import datetime

import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.svm import SVR
from sklearn.metrics import root_mean_squared_error, r2_score
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb

In [ ]:
#Abrir dataset original
estacion = "MX"

path = f"D:/Josefina/Proyectos/ProyectoChile/{estacion}/proceed/merge_tot/{estacion}_merge_comp.csv"
data_com = pd.read_csv(path)

#Revisar nombres de columnas
print(data_com.columns)


Index(['Unnamed: 0', 'X', 'ID', 'date', 'estacion', 'PM25', 'AOD_055', 'ndvi',
       'BCSMASS_dia', 'DUSMASS_dia', 'DUSMASS25_dia', 'OCSMASS_dia',
       'SO2SMASS_dia', 'SO4SMASS_dia', 'SSSMASS_dia', 'SSSMASS25_dia',
       'blh_mean', 'blh_min', 'blh_max', 'blh_sd', 'sp_mean', 'sp_min',
       'sp_max', 'sp_sd', 'd2m_mean', 'd2m_min', 'd2m_max', 'd2m_sd',
       't2m_mean', 't2m_min', 't2m_max', 't2m_sd', 'v10_mean', 'v10_min',
       'v10_max', 'v10_sd', 'u10_mean', 'u10_min', 'u10_max', 'u10_sd',
       'tp_mean', 'tp_min', 'tp_max', 'tp_sd', 'DEM', 'dayWeek'],
      dtype='object')


In [6]:
#Formato de la columna date
data_com["date"] = pd.to_datetime(data_com["date"], format="%Y-%m-%d")
#Crear columna dia de la semana
data_com["dayWeek"] = data_com["date"].dt.weekday + 1
# Columan de los años 
data_com["year"] = data_com["date"].dt.year

# Ver años disponibles
print(data_com["year"].unique())

# Filtrar (excluir 2024)
data_com = data_com[data_com["year"] != 2024]

# Verificación
print(data_com["year"].unique())


[2015 2016 2017 2018 2019 2020 2021 2022 2023 2024]
[2015 2016 2017 2018 2019 2020 2021 2022 2023]


In [7]:
#Definir vatriables, ver cuales se usan en cada ciudad
variables = [
    "AOD_055", "ndvi", "BCSMASS_dia", "DUSMASS_dia", "OCSMASS_dia",
    "SO2SMASS_dia", "SO4SMASS_dia", "SSSMASS_dia",
    "blh_mean", "sp_mean", "d2m_mean", "t2m_mean",
    "v10_mean", "u10_mean", "tp_mean", "DEM", "dayWeek"
]

y = data_com["PM25"]
X = data_com[variables]


In [ ]:
#Regresion lineal 
X = sm.add_constant(X)
modelo = sm.OLS(y, X, missing="drop").fit()
print(modelo.summary())


                            OLS Regression Results                            
Dep. Variable:                   PM25   R-squared:                       0.480
Model:                            OLS   Adj. R-squared:                  0.480
Method:                 Least Squares   F-statistic:                     1324.
Date:                Wed, 21 Jan 2026   Prob (F-statistic):               0.00
Time:                        12:23:53   Log-Likelihood:                -78232.
No. Observations:               22963   AIC:                         1.565e+05
Df Residuals:                   22946   BIC:                         1.566e+05
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const           65.8198      9.032      7.287   

In [ ]:
# Calcular VIF
vif_df = pd.DataFrame()
vif_df["Variable"] = X.columns
vif_df["VIF"] = [
    variance_inflation_factor(X.values, i)
    for i in range(X.shape[1])
]
vif_df_sorted = vif_df.sort_values("VIF", ascending=False)
print(vif_df_sorted)
a = vif_df_sorted.copy()


Parametros del VIF
- VIF < 5 → OK
- 5 ≤ VIF < 10 → colinealidad moderada
- VIF ≥ 10 → problema serio


In [14]:
def evaluar_modelo(modelo, X_test, y_test, tipo_modelo="SVR"):
    # Predicciones
    predicciones = modelo.predict(X_test)

    # Eliminar predicciones negativas (como en R)
    mask = predicciones > 0
    pred = predicciones[mask]
    real = y_test[mask]

    # Métricas
    r2 = np.corrcoef(pred, real)[0, 1] ** 2
    pearson = np.corrcoef(pred, real)[0, 1]
    rmse = root_mean_squared_error(real, pred)
    bias = np.mean(pred - real)

    resultados = pd.DataFrame({
        "R2": [round(r2, 5)],
        "Pearson": [round(pearson, 3)],
        "RMSE": [round(rmse, 3)],
        "Bias": [round(bias, 3)],
        "Min_Pred": [round(pred.min(), 3)],
        "Max_Pred": [round(pred.max(), 3)]
    })

    return resultados


In [ ]:
### -------------- SVR --------------------------
# Datos de entrada
estacion = "MX"
modelo_id = "1"

base_dir = f"D:/Josefina/Proyectos/ProyectoChile/{estacion}/modelos/ParticionDataSet/"

train_data = pd.read_csv(f"{base_dir}/Modelo_{modelo_id}/M{modelo_id}_train_{estacion}.csv")
test_data  = pd.read_csv(f"{base_dir}/Modelo_{modelo_id}/M{modelo_id}_test_{estacion}.csv")

features = [
    "AOD_055", "ndvi", "BCSMASS_dia", "DUSMASS_dia",
    "SO2SMASS_dia", "SO4SMASS_dia", "SSSMASS_dia",
    "blh_mean", "sp_mean", "d2m_mean",
    "v10_mean", "u10_mean", "tp_mean", "DEM", "dayWeek"
]

X_train = train_data[features]
y_train = train_data["PM25"]

X_test = test_data[features]
y_test = test_data["PM25"]

#Construccion del modelo
svr_model = SVR(
    kernel="rbf",
    C=10,
    epsilon=0.1
)
#Ajuste del modelo
svr_model.fit(X_train, y_train)

# Evaluar el modelo
resultados_SVR = evaluar_modelo(
    modelo=svr_model,
    X_test=X_test,
    y_test=y_test,
    tipo_modelo="SVR"
)

#Resultados
print(resultados_SVR)



        R2  Pearson   RMSE   Bias  Min_Pred  Max_Pred
0  0.07389    0.272  9.845 -1.352    18.198    23.038


In [18]:
### -------------- ExtraTrees --------------------------

estacion = "BA"
modelo = "1"

dir_path = f"D:/Josefina/Proyectos/ProyectoChile/{estacion}/modelos/ParticionDataSet/"

train_data = pd.read_csv(
    f"{dir_path}/Modelo_{modelo}/M{modelo}_train_{estacion}.csv"
)
test_data = pd.read_csv(
    f"{dir_path}/Modelo_{modelo}/M{modelo}_test_{estacion}.csv"
)
#Definir variables
features = [
    "AOD_055", "ndvi", "BCSMASS_dia", "DUSMASS_dia", "t2m_mean",
    "SO2SMASS_dia", "SO4SMASS_dia", "SSSMASS_dia", "blh_mean", "sp_mean",
    "d2m_mean", "v10_mean", "u10_mean", "tp_mean", "DEM", "dayWeek"
]

X_train = train_data[features]
y_train = train_data["PM25"]

X_test = test_data[features]
y_test = test_data["PM25"]

#Construir modelo
et_model = ExtraTreesRegressor(
    n_estimators=500,      # similar a ranger por defecto
    max_features=5,        # mtry
    min_samples_leaf=5,    # min.node.size
    bootstrap=False,       # Extra Trees puro
    random_state=123,
    n_jobs=-1
)
#Ajustar al modelo
et_model.fit(X_train, y_train)

pred = et_model.predict(X_test)

# Filtrar predicciones negativas, revisar/probar si lo hacemos ono
mask = pred > 0
pred = pred[mask]
real = y_test.values[mask]

r2 = np.corrcoef(real, pred)[0, 1] ** 2
pearson = np.corrcoef(real, pred)[0, 1]
rmse = root_mean_squared_error(real, pred)
bias = np.mean(pred - real)

resultados_ET = pd.DataFrame({
    "R2": [round(r2, 5)],
    "Pearson": [round(pearson, 3)],
    "RMSE": [round(rmse, 3)],
    "Bias": [round(bias, 3)],
    "Min_Pred": [round(pred.min(), 3)],
    "Max_Pred": [round(pred.max(), 3)]
})

print(resultados_ET)


# import joblib

# save_path = f"D:/Josefina/Proyectos/Tesis/{estacion}/modelos/"
# joblib.dump(
#     et_model,
#     f"{save_path}/01-ET-M{modelo}-170625-{estacion}.joblib"
# )


        R2  Pearson   RMSE   Bias  Min_Pred  Max_Pred
0  0.57849    0.761  7.195  0.108     6.386    47.524


In [21]:
### -------------- RANDOM FOREST --------------------------
# Datos de entrada
estacion = "MX"
modelo_id = "1"

base_dir = f"D:/Josefina/Proyectos/ProyectoChile/{estacion}/modelos/ParticionDataSet/"

train_data = pd.read_csv(f"{base_dir}/Modelo_{modelo_id}/M{modelo_id}_train_{estacion}.csv")
test_data  = pd.read_csv(f"{base_dir}/Modelo_{modelo_id}/M{modelo_id}_test_{estacion}.csv")

features = [
    "AOD_055", "ndvi", "BCSMASS_dia", "DUSMASS_dia",
    "SO2SMASS_dia", "SO4SMASS_dia", "SSSMASS_dia",
    "blh_mean", "sp_mean", "d2m_mean",
    "v10_mean", "u10_mean", "tp_mean", "DEM", "dayWeek"
]

X_train = train_data[features]
y_train = train_data["PM25"]

X_test = test_data[features]
y_test = test_data["PM25"]

#Construccion del modelo
rf_model = RandomForestRegressor(
    n_estimators=500,      # caret suele usar 500
    random_state=123,
    n_jobs=-1
)

rf_model.fit(X_train, y_train)


pred = rf_model.predict(X_test)

# Filtrar predicciones negativas (igual que en R)
mask = pred > 0
pred = pred[mask]
real = y_test.values[mask]

r2 = np.corrcoef(real, pred)[0, 1] ** 2
pearson = np.corrcoef(real, pred)[0, 1]
rmse = root_mean_squared_error(real, pred)
bias = np.mean(pred - real)

resultados_RF = pd.DataFrame({
    "R2": [round(r2, 5)],
    "Pearson": [round(pearson, 3)],
    "RMSE": [round(rmse, 3)],
    "Bias": [round(bias, 3)],
    "Min_Pred": [round(pred.min(), 3)],
    "Max_Pred": [round(pred.max(), 3)]
})

print(resultados_RF)
importances = pd.DataFrame({
    "Variable": features,
    "Importance": rf_model.feature_importances_
}).sort_values("Importance", ascending=False)

print(importances)


        R2  Pearson   RMSE  Bias  Min_Pred  Max_Pred
0  0.69809    0.836  5.528  0.16     5.099    92.055
        Variable  Importance
0        AOD_055    0.203057
1           ndvi    0.098273
2    BCSMASS_dia    0.088197
9       d2m_mean    0.087477
4   SO2SMASS_dia    0.080688
12       tp_mean    0.076351
7       blh_mean    0.069875
3    DUSMASS_dia    0.054615
10      v10_mean    0.053352
6    SSSMASS_dia    0.041569
11      u10_mean    0.038370
5   SO4SMASS_dia    0.037890
8        sp_mean    0.037628
13           DEM    0.019693
14       dayWeek    0.012964


In [ ]:
### -------------- XGB --------------------------
# Datos de entrada
estacion = "MX"
modelo_id = "1"

base_dir = f"D:/Josefina/Proyectos/ProyectoChile/{estacion}/modelos/ParticionDataSet/"

train_data = pd.read_csv(f"{base_dir}/Modelo_{modelo_id}/M{modelo_id}_train_{estacion}.csv")
test_data  = pd.read_csv(f"{base_dir}/Modelo_{modelo_id}/M{modelo_id}_test_{estacion}.csv")

features = [
    "AOD_055", "ndvi", "BCSMASS_dia", "DUSMASS_dia",
    "SO2SMASS_dia", "SO4SMASS_dia", "SSSMASS_dia",
    "blh_mean", "sp_mean", "d2m_mean",
    "v10_mean", "u10_mean", "tp_mean", "DEM", "dayWeek"
]

X_train = train_data[features]
y_train = train_data["PM25"]

X_test = test_data[features]
y_test = test_data["PM25"]

#Colocar en una matrix. Esto se hizo en el r pero es necesario en python?
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

#Hiperparametros seteados, pueden variar segun el centro urbano
params = {
    "booster": "gbtree",
    "objective": "reg:squarederror",
    "eval_metric": "rmse",
    "eta": 0.3,
    "max_depth": 6,
    "gamma": 0,
    "subsample": 0.8,
    "colsample_bytree": 1.0,
    "min_child_weight": 1
}
#Entrenar modelo
xgb_model = xgb.train(
    params=params,
    dtrain=dtrain,
    num_boost_round=2000
)
#Predicciom
y_pred = xgb_model.predict(dtest)

# eliminar predicciones negativas (igual que R)
mask = y_pred > 0
y_pred = y_pred[mask]
y_true = y_test.values[mask]

#Resultados
r2 = np.corrcoef(y_pred, y_true)[0, 1] ** 2
pearson = np.corrcoef(y_pred, y_true)[0, 1]
rmse = np.sqrt(np.mean((y_pred - y_true) ** 2))
bias = np.mean(y_pred - y_true)

resultados_XGB = pd.DataFrame({
    "R2": [round(r2, 5)],
    "Pearson": [round(pearson, 3)],
    "RMSE": [round(rmse, 3)],
    "Bias": [round(bias, 3)],
    "Min_Pred": [round(y_pred.min(), 3)],
    "Max_Pred": [round(y_pred.max(), 3)]
})

print(resultados_XGB)


        R2  Pearson   RMSE   Bias  Min_Pred    Max_Pred
0  0.74702    0.864  5.012  0.092     0.436  106.114998
